# CARTOframes with Dask

This notebook recreate the workflow from <https://jakevdp.github.io/blog/2015/08/14/out-of-core-dataframes-in-python/>, where the author explores [`dask`](http://dask.pydata.org/en/latest/) for splitting up the computations across multiple cores in a machine to complete tasks more quickly. 

## Basics

You'll need the following for this:

1. Your CARTO username
2. Your API key

Paste these values in the quotes (`''`) below.

In [ ]:
import pandas as pd
import cartoframes

username = ''   # <-- insert your username here
api_key = ''    # <-- insert your API key here
tablename = ''  # <-- insert your tablename here

cc = cartoframes.CartoContext('https://{}.carto.com/'.format(username),
                              api_key)

In [ ]:
from dask import dataframe as dd
import pandas as pd
columns = ["name", "amenity", "Longitude", "Latitude"]
data = dd.read_csv('POIWorld.csv', usecols=columns)

In [ ]:
with_name = data[data.name.notnull()]
with_amenity = data[data.amenity.notnull()]

is_starbucks = with_name.name.str.contains('[Ss]tarbucks')
is_dunkin = with_name.name.str.contains('[Dd]unkin')

starbucks = with_name[is_starbucks].compute()
dunkin = with_name[is_dunkin].compute()

In [ ]:
starbucks['type'] = 'starbucks'
dunkin['type'] = 'dunkin'
coffee_places = pd.concat([starbucks, dunkin])
coffee_places.head(20)

## Write DataFrame to CARTO

In [ ]:
# specify columns for lng/lat so carto will create a geometry
cc.write(coffee_places,
         table_name='coffee_places',
         lnglat=('longitude', 'latitude'))

### Let's visualize this DataFrame

Category map on Dunkin' Donuts vs. Starbucks (aka, color by 'type')

In [ ]:
from cartoframes import Layer
cc.map(layers=Layer('coffee_places', color='type', size=5),
       zoom=9, lng=-71.0637, lat=36.4275,
       interactive=False)

## Fast Food

In [ ]:
is_fastfood = with_amenity.amenity.str.contains('fast_food')
fastfood = with_amenity[is_fastfood]
fastfood.name.value_counts().head(12)

In [ ]:
ff = fastfood.compute()
cc.write(ff,
         tablename='fastfood_dask',
         lnglat=('longitude', 'latitude'))

### Number of Fast Food places in this OSM dump

In [ ]:
len(ff)

### Recreating the map from the blog

In [ ]:
cc.map(layers=Layer('fastfood_dask', size=2, color='#FFF'))

### Adding measures from the Data Observatory

We can augment our datasets to find out some demographics at the areas of each of the coffee places if you wanted to add features for building a model.

In [ ]:
# DO measures: Total Population,
#              Children under 18 years of age
#              Median income

data_obs_measures = [{'numer_id': 'us.census.acs.B01003001'},
                     {'numer_id': 'us.census.acs.B17001001'},
                     {'numer_id': 'us.census.acs.B19013001'}]
coffee_augmented = cc.data('coffee_places', data_obs_measures)
coffee_augmented.head()